In [1]:
import os
import time
import cv2

from sklearn.model_selection import train_test_split
from glob import glob
from PIL import Image
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt
import torch

import torch.nn as nn
import torch.optim as optim

from torchvision import datasets, models, transforms


In [2]:
img_list = glob('/nas/data/dacon/open/crop_img/**/*.png')
label_list = ['_'.join(ann.split('_')[:-1]).split('/')[-1] for ann in img_list]

X_train, X_val, y_train, y_val = train_test_split(img_list, label_list, test_size=0.3, random_state=42)

# Train 데이터와 Validation 데이터의 크기 확인
print('Train 데이터 수:', len(X_train))
print('Validation 데이터 수:', len(X_val))

Train 데이터 수: 11900
Validation 데이터 수: 5100


In [3]:
import shutil

for i in tqdm(X_train):
    os.makedirs('/'.join(i.split('/')[:-1]).replace("crop_img/","crop_img/train/"), exist_ok=True)
    shutil.copy(i, i.replace("crop_img/","crop_img/train/"))
for i in tqdm(X_val):
    os.makedirs('/'.join(i.split('/')[:-1]).replace("crop_img/","crop_img/val/"), exist_ok=True)
    shutil.copy(i, i.replace("crop_img/","crop_img/val/"))

100%|██████████| 5100/5100 [00:33<00:00, 152.57it/s]


In [2]:
# ddir = '/data/hym_data'
ddir = '/nas/data/dacon/open/crop_img'

batch_size = 16
num_workers = 8

def custom_loader(path):
    img = Image.open(path)
    return img

data_transformers = {
    'train': transforms.Compose(
        [
         transforms.RandomResizedCrop(224), 
         transforms.RandomHorizontalFlip(),
         transforms.ToTensor(),
         transforms.Normalize([0.490, 0.449, 0.411], [0.231, 0.221, 0.230])
        ]
    ),
    'val': transforms.Compose(
        [
         transforms.Resize(128),
         transforms.CenterCrop(224),
         transforms.ToTensor(),
         transforms.Normalize([0.490, 0.449, 0.411],[0.231, 0.221, 0.230])
        ]
    )
}

img_data = {
    k: datasets.ImageFolder(os.path.join(ddir, k), data_transformers[k])
    for k in ['train','val']
}
dloaders = {
    k: torch.utils.data.DataLoader(
        img_data[k], batch_size=batch_size, shuffle=True, num_workers=num_workers
    )
    for k in ['train', 'val']
}
dset_sizes = {x: len(img_data[x]) for x in ['train', 'val']}
classes = img_data['train'].classes

dvc = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [6]:
dataset = img_data['train']
# print("Classes:", dataset.classes)
# print("Class-to-index mapping:", dataset.class_to_idx)
# print("Number of samples:", len(dataset))
# print("Sample paths and labels:")
for img_path, label in tqdm(dataset.imgs):
    try:
        aa = cv2.imread(img_path)
        bb = Image.open(img_path)
        if aa is None or bb is None:
            print(img_path)
    except Exception as e:
        print(img_path)
        print(e)
        print(img_path,'이거 문제입니다.')
        print(11111)
print('\ndone')

libpng error: Read Error


/nas/data/dacon/open/crop_img/train/ssangyong_korando_suv_2019_2020/ssangyong_korando_suv_2019_2020_18.png

done


In [3]:
cfg = {
    'vgg16' : [64, 64, 'M', 128, 128, 'M', 256, 256, 256,      'M', 512, 512, 512,      'M', 512, 512, 512,      'M'],
    'vgg19' : [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M']
}

class VGG(nn.Module):

    def __init__(self, features, num_classes=34):
        super().__init__()
        self.features = features

        self.classifier = nn.Sequential(
            nn.Linear(512*7*7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        output = self.features(x)
        output = output.view(output.size()[0], -1)
        output = self.classifier(output)

        return output

def make_layers(cfg):
    layers = []

    input_channel = 3
    for l in cfg:
        if l == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            continue

        layers += [nn.Conv2d(input_channel, l, kernel_size=3, padding=1)]

       
        layers += [nn.BatchNorm2d(l)]

        layers += [nn.ReLU(inplace=True)]
        input_channel = l

    return nn.Sequential(*layers)


def vgg16(num=34):
    return VGG(make_layers(cfg['vgg16']),num_classes=num)

def vgg19(num=34):
    return VGG(make_layers(cfg['vgg19']),num_classes=num)



In [4]:
def train(model, loss_func, optimizer, epochs=10):
   start = time.time()

   accuracy = 0.0

   for e in range(epochs):
        print(f'Epoch number {e+1}/{epochs}')
        print('=' * 20)

        for dset in ['train', 'val']:
            if dset == 'train':
                model.train()  
            else:
                model.eval() 

            loss = 0.0
            successes = 0
            total_examples = 0

            for imgs, tgts in tqdm(dloaders[dset]):
                try:
                    imgs = imgs.to(dvc)
                    tgts = tgts.to(dvc)
                    optimizer.zero_grad()
                except OSError as e:
                    # 오류가 발생한 이미지의 경로를 출력하거나 원하는 방식으로 기록할 수 있습니다.
                    print("오류가 발생한 이미지 경로:", imgs)
                    break

                with torch.set_grad_enabled(dset == 'train'):
                    ops = model(imgs)
                    _, preds = torch.max(ops, 1)
                    loss_curr = loss_func(ops, tgts)

                    if dset == 'train':
                        loss_curr.backward()
                        optimizer.step()

                loss += loss_curr.item() * imgs.size(0)
                successes += torch.sum(preds == tgts.data)
                total_examples += imgs.size(0)
                # print(f'loss : {loss}, successes : {successes}, total_examples : {total_examples}')

            loss_epoch = loss / total_examples
            accuracy_epoch = successes.double() / total_examples

            print(f'{dset} loss in this epoch: {loss_epoch}, accuracy in this epoch: {accuracy_epoch}')
            if dset == 'val' and accuracy_epoch > accuracy:
                accuracy = accuracy_epoch

   time_delta = time.time() - start
   print(f'Training finished in {time_delta // 60}mins {time_delta % 60}secs')
   print(f'Best validation set accuracy: {accuracy}')

   return model

In [5]:
model = vgg19()
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
model.to(dvc)
pretrained_model = train(model, loss_func, optimizer, epochs=50)


Epoch number 1/50


100%|██████████| 850/850 [01:13<00:00, 11.52it/s]


train loss in this epoch: 3.332436030748548, accuracy in this epoch: 0.0689021251562615


100%|██████████| 213/213 [00:06<00:00, 33.52it/s]


val loss in this epoch: 4.152553351907169, accuracy in this epoch: 0.027352941176470587
Epoch number 2/50


...

100%|██████████| 850/850 [01:13<00:00, 11.50it/s]


train loss in this epoch: 0.3830533381170045, accuracy in this epoch: 0.8833002426649019


100%|██████████| 213/213 [00:06<00:00, 33.42it/s]


val loss in this epoch: 4.054973110872156, accuracy in this epoch: 0.08029411764705882
Epoch number 28/50


 20%|█▉        | 167/850 [00:14<00:59, 11.57it/s]

In [ ]:
def test(model=pretrained_model):

  correct_pred = {classname: 0 for classname in classes}
  total_pred = {classname: 0 for classname in classes}


  with torch.no_grad():
      for images, labels in dloaders['val']:


          images = images.to(dvc)
          labels = labels.to(dvc)
        
          outputs = model(images)
          _, predictions = torch.max(outputs, 1)

          for label, prediction in zip(labels, predictions):
              if label == prediction:
                  correct_pred[classes[label]] += 1
              total_pred[classes[label]] += 1



  for classname, correct_count in correct_pred.items():
      accuracy = 100 * float(correct_count) / total_pred[classname]
      print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

In [ ]:
test(pretrained_model)

In [ ]:
def imageshow(img, text=None):
  img = img.numpy().transpose((1,2,0))
  avg = np.array([0.490, 0.449, 0.411])
  stddev = np.array([0.231, 0.221, 0.230])
  img = stddev * img + avg
  img = np.clip(img, 0, 1)
  plt.imshow(img)
  if text is not None:
    plt.title(text)

In [ ]:
def visualize_predictions(pretrained_model, max_num_imgs=4):
    torch.manual_seed(1)
    was_model_training = pretrained_model.training
    pretrained_model.eval()
    imgs_counter = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (imgs, tgts) in enumerate(dloaders['val']):
            imgs = imgs.to(dvc)
            tgts = tgts.to(dvc)
            ops = pretrained_model(imgs)
            _, preds = torch.max(ops, 1)
            
            for j in range(imgs.size()[0]):
                imgs_counter += 1
                ax = plt.subplot(max_num_imgs//2, 2, imgs_counter)
                ax.axis('off')
                ax.set_title(f'pred: {classes[preds[j]]} || target: {classes[tgts[j]]}')
                imageshow(imgs.cpu().data[j])

                if imgs_counter == max_num_imgs:
                    pretrained_model.train(mode=was_model_training)
                    return
        pretrained_model.train(mode=was_model_training)

In [ ]:
visualize_predictions(pretrained_model)

In [ ]:
from tqdm import tqdm
from glob import glob
import shutil

